In [1]:
import asyncio
import io
import glob
import os
import sys
import time
import uuid
import requests
from urllib.parse import urlparse
from io import BytesIO
# To install this module, run:
# python -m pip install Pillow
from PIL import Image, ImageDraw
from azure.cognitiveservices.vision.face import FaceClient
from msrest.authentication import CognitiveServicesCredentials
from azure.cognitiveservices.vision.face.models import TrainingStatusType, Person
from azure.cognitiveservices.vision.face.models import FaceAttributeType

In [2]:
#Retrive from Face Resource - Resource Management
KEY = "1ab3848ea22649299da961e8feb813a4"
ENDPOINT = "https://face-rec-app.cognitiveservices.azure.com/"

In [3]:
# Create an authenticated FaceClient.
face_client = FaceClient(ENDPOINT, CognitiveServicesCredentials(KEY))

In [4]:
'''
Detect face(s) with attributes in a URL image
'''
# Image of face(s)
face1_url = 'https://upload.wikimedia.org/wikipedia/commons/f/f2/Kristofer_Hivju_%28Cropped%2C_2015%29.jpg'
face1_name = os.path.basename(face1_url)

# List of url images
url_images = [face1_url]

In [5]:
# Attributes you want returned with the API call, a list of FaceAttributeType enum (string format)
face_attributes = ['headPose', 'glasses', 'occlusion', 'blur', 'exposure', 'noise']

In [6]:
# Detect a face with attributes, returns a list[DetectedFace]
for image in url_images:
    detected_faces = face_client.face.detect_with_url(url=image, return_face_attributes=face_attributes)
    if not detected_faces:
        raise Exception(
            'No face detected from image {}'.format(os.path.basename(image)))

In [7]:
    '''
    Display the detected face with attributes and bounding box
    '''
    # Face IDs are used for comparison to faces (their IDs) detected in other images.
    for face in detected_faces:
        print()
        print('Detected face ID from', os.path.basename(image), ':')
        # ID of detected face
        print(face.face_id)
        # Show all facial attributes from the results
        print()
        print('Facial attributes detected:')
        print('Head pose: ', face.face_attributes.head_pose)
        print('Glasses: ', face.face_attributes.glasses)
        print('Occlusion: ')
        print('\teye_occluded: ', face.face_attributes.occlusion.eye_occluded)
        print('\tforehead_occluded: ', face.face_attributes.occlusion.forehead_occluded)
        print('Blur: ', face.face_attributes.blur)
        print('Exposure: ', face.face_attributes.exposure)
        print('Noise: ', face.face_attributes.noise)
        print()



Detected face ID from Kristofer_Hivju_%28Cropped%2C_2015%29.jpg :
8383822b-15ff-4bbe-ade9-f65eb328227b

Facial attributes detected:
Head pose:  {'additional_properties': {}, 'roll': -9.4, 'yaw': 8.9, 'pitch': 12.7}
Glasses:  GlassesType.no_glasses
Occlusion: 
	eye_occluded:  False
	forehead_occluded:  False
Blur:  {'additional_properties': {}, 'blur_level': <BlurLevel.low: 'Low'>, 'value': 0.24}
Exposure:  {'additional_properties': {}, 'exposure_level': <ExposureLevel.over_exposure: 'OverExposure'>, 'value': 0.75}
Noise:  {'additional_properties': {}, 'noise_level': <NoiseLevel.medium: 'Medium'>, 'value': 0.38}



In [8]:
    # Convert width height to a point in a rectangle
    def getRectangle(faceDictionary):
        rect = faceDictionary.face_rectangle
        left = rect.left
        top = rect.top
        right = left + rect.width
        bottom = top + rect.height

        return ((left, top), (right, bottom))

In [9]:
    # Download the image from the url, so can display it in popup/browser
    response = requests.get(image)
    img = Image.open(BytesIO(response.content))

In [10]:
    # For each face returned use the face rectangle and draw a red box.
    print('Drawing rectangle around face... see popup for results.')
    print()
    draw = ImageDraw.Draw(img)
    for face in detected_faces:
        draw.rectangle(getRectangle(face), outline='red')

Drawing rectangle around face... see popup for results.



In [11]:
    # Display the image in the users default image browser.
    img.show()

In [12]:
# Extract the head_pose data 

print(face.face_attributes.head_pose)

{'additional_properties': {}, 'roll': -9.4, 'yaw': 8.9, 'pitch': 12.7}


In [13]:
# Use rectangles to see if face is centred 
w,h=img.size    # w=Width and h=Height
print("Width =",w,end="\t")
print("Height =",h)

Width = 250	Height = 600


In [14]:
    def RectCentre(faceDictionary, img):
        rect = faceDictionary.face_rectangle
        w,h=img.size
        left = rect.left
        top = rect.top
        right = left + rect.width
        bottom = top + rect.height
        rightspace = w - right
        bottomspace = h - bottom
        
        if left != rightspace: 
            print('Face is not horizontally centred')
            
        if top != bottomspace:
            print('Face is not vertically centred')
        

In [15]:
RectCentre(face, img)

Face is not horizontally centred
Face is not vertically centred
